In [1]:
from ortools.sat.python import cp_model

In [2]:
p = 2**31-1

def restrict(model, expr, lb, ub):
    model.Add(lb <= expr)
    model.Add(expr <= ub)

def model_copy(model, u, v0, v1):
    e = model.NewBoolVar("")
    model.Add(e*(p-1) + u == v0 + v1)
    restrict(model, v0+v1-e*p, 0, p-1)

def model_square(model, u, v):
    e = model.NewBoolVar("")
    model.Add(e*(p-1) + u == 2*v)
    restrict(model, 2*v-e*p, 0, p-1)
    
def model_addition(model, u0, u1, v):
    model.Add(v == u0 + u1)

def model_constant_addition(model, u, v):
    model.Add(v >= u)

In [3]:
def model_round(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    # copy left branch (branches enumerated from left to right)
    e = [model.NewIntVar(0, p-1, "") for _ in range(18)]
    model_copy(model, u[0], v[2], e[0])
    model_copy(model, u[1], v[3], e[1])
    # round function
    model_constant_addition(model, e[1], e[2])
    model_copy(model, e[2], e[3], e[4]) 
    model_square(model, e[3], e[5])
    model_addition(model, e[0], e[5], e[6])
    model_copy(model, e[6], e[7], e[8]) 
    model_addition(model, e[4], e[7], e[9])
    model_copy(model, e[9], e[10], e[11]) 
    model_addition(model, e[8], e[10], e[12])
    model_constant_addition(model, e[11], e[13])
    model_copy(model, e[13], e[14], e[15]) 
    model_square(model, e[14], e[16])
    model_addition(model, e[12], e[16], e[17])
    model_addition(model, e[17], u[3], v[1])
    model_addition(model, e[15], u[2], v[0])

In [4]:
def model_step(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    exponents = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] + [v]

    # key additions
    for i in range(4):
        model_constant_addition(model, u[i], exponents[0][i])

    # rounds
    for i in range(4):
        model_round(model, exponents[i], exponents[i+1])

# degree estimations

In [5]:
# total degree
for r in range(1, 36):
    nsteps = r//4
    extra_rounds = r%4
    res = []
    for a in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == a:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        model.Maximize(sum(us[0]))
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL:
            res.append(int(solver.ObjectiveValue()))
        elif status == cp_model.FEASIBLE:
            print(a, "error")
        else:
            print(a, "error")
    print(r, min(res), max(res), res)

1 1 4 [2, 4, 1, 1]
2 2 16 [8, 16, 2, 4]
3 8 64 [32, 64, 8, 16]
4 32 256 [128, 256, 32, 64]
5 128 1024 [512, 1024, 128, 256]
6 512 4096 [2048, 4096, 512, 1024]
7 2048 16384 [8192, 16384, 2048, 4096]
8 8192 65536 [32768, 65536, 8192, 16384]
9 32768 262144 [131072, 262144, 32768, 65536]
10 131072 1048576 [524288, 1048576, 131072, 262144]
11 524288 4194304 [2097152, 4194304, 524288, 1048576]
12 2097152 16777216 [8388608, 16777216, 2097152, 4194304]
13 8388608 67108864 [33554432, 67108864, 8388608, 16777216]
14 33554432 268435456 [134217728, 268435456, 33554432, 67108864]
15 134217728 1073741824 [536870912, 1073741824, 134217728, 268435456]
16 536870912 3221225470 [2147483647, 3221225470, 536870912, 1073741824]
17 2147483647 5368709115 [4294967293, 5368709115, 2147483647, 3221225470]
18 4294967293 6979321850 [5905580027, 6979321850, 4294967293, 5368709115]
19 5905580027 8053063672 [7516192761, 8053063672, 5905580027, 6979321850]
20 7516192761 8455716856 [8321499128, 8455716856, 7516192761, 

In [6]:
# univariate degree
for r in range(1, 20):
    nsteps = r//4
    extra_rounds = r%4
    res = []
    for a in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == a:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        univariate_deg = model.NewIntVar(0, p-1, "")
        model.AddMaxEquality(univariate_deg, us[0])
        model.Maximize(univariate_deg)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL:
            res.append(int(solver.ObjectiveValue()))
        elif status == cp_model.FEASIBLE:
            print(a, "error")
        else:
            print(a, "error")
    print(r, min(res), max(res), res)

1 1 4 [2, 4, 1, 1]
2 2 16 [8, 16, 2, 4]
3 8 64 [32, 64, 8, 16]
4 32 256 [128, 256, 32, 64]
5 128 1024 [512, 1024, 128, 256]
6 512 4096 [2048, 4096, 512, 1024]
7 2048 16384 [8192, 16384, 2048, 4096]
8 8192 65536 [32768, 65536, 8192, 16384]
9 32768 262144 [131072, 262144, 32768, 65536]
10 131072 1048576 [524288, 1048576, 131072, 262144]
11 524288 4194304 [2097152, 4194304, 524288, 1048576]
12 2097152 16777216 [8388608, 16777216, 2097152, 4194304]
13 8388608 67108864 [33554432, 67108864, 8388608, 16777216]
14 33554432 268435456 [134217728, 268435456, 33554432, 67108864]
15 134217728 1073741824 [536870912, 1073741824, 134217728, 268435456]
16 536870912 2147483646 [2147483646, 2147483646, 536870912, 1073741824]
17 2147483646 2147483646 [2147483646, 2147483646, 2147483646, 2147483646]
18 2147483646 2147483646 [2147483646, 2147483646, 2147483646, 2147483646]
19 2147483646 2147483646 [2147483646, 2147483646, 2147483646, 2147483646]


# finding max-data properties

In [7]:
# check how far we can get
nsteps = 9
extra_rounds = 0
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(4):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 2 zero sum


In [8]:
# check how far we can get
nsteps = 9
extra_rounds = 1
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(4):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

In [9]:
# first round with a dense component function, the second component function is dense
nsteps = 8
extra_rounds = 3
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(4):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 0 zero sum
0 2 zero sum
0 3 zero sum
1 2 zero sum
1 3 zero sum
2 2 zero sum


# finding minimal data properties
We go down from 36 rounds

In [10]:
# check positions
nsteps = 5
extra_rounds = 1
eligible_positions = set()
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(4):
            if i == a:
                e = model.NewIntVar(1, 2, "")
                model.Add(us[0][i] == ((p-1)//2)*e) # subgroup of squares
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(4):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            pass
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
            eligible_positions.add((a,b))
        else:
            print(a, b, "error")

0 2 zero sum


In [11]:
# check how low the data can go
mindegs = [1073741823, 715827882, 357913941, 306783378, 238609294, 195225786, 153391689, 119304647, 102261126, 97612893, 69273666, 65075262, 51130563, 34636833, 34087042, 32537631, 27889398, 23091222, 21691754, 17043521, 14221746, 13944699, 11545611, 10845877, 9896238, 9296466, 7697074, 7110873, 6487866, 6297606, 4948119, 4740582, 4648233, 3848537, 3298746, 3243933, 3148803, 3098822, 2370291, 2162622, 2099202, 2031678, 1649373, 1580194, 1549411, 1292886, 1099582, 1081311, 1049601, 1015839, 926838, 899658, 790097, 720874, 699734, 677226, 646443, 589806, 549791, 463419, 458766, 449829, 430962, 360437, 349867, 338613, 308946, 299886, 294903, 229383, 225742, 215481, 209286, 196602, 184698, 154473, 152922, 149943, 143654, 112871, 104643, 102982, 99962, 98301, 92349, 84258, 76461, 71827, 69762, 65538, 65534, 61566, 51491, 50974, 49981, 42966, 42129, 41706, 34881, 32769, 32767, 30783, 29898, 28086, 25487, 23254, 21846, 21483, 20853, 20522, 19026, 14949, 14322, 14043, 13902, 11627, 10923, 10261, 9966, 9513, 9362, 7282, 7161, 6951, 6342, 6138, 5958, 4983, 4774, 4681, 4634, 3906, 3641, 3322, 3171, 3069, 2979, 2718, 2387, 2317, 2114, 2046, 1986, 1953, 1661, 1386, 1359, 1302, 1057, 1023, 993, 906, 693, 682, 662, 651, 558, 462, 453, 434, 341, 331, 302, 279, 231, 217, 198, 186, 154, 151, 126, 99, 93, 77, 66, 63, 62, 42, 33, 31, 22, 21, 18, 14, 11, 9, 7, 6, 3, 2, 1, 0]
for a, b in eligible_positions:
    for d in mindegs:
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        
        # input constraints
        for i in range(16):
            if i == a:
                model.Add(us[0][i] >= d) # subgroup of squares
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            model_step(model, us[i], us[i+1])
        for i in range(16):
            model_constant_addition(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            model_round(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(16):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 32
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # print(a, b, "trail exists")
            # for u in us:
            #     print([solver.Value(x) for x in u])
            print(a, b, d, "fail")
            break
        elif status == cp_model.INFEASIBLE:
            print(a, b, d, "zero sum")
        else:
            print(a, b, "error")
# so each position can be reduced to 0 data

0 2 1073741823 zero sum
0 2 715827882 zero sum
0 2 357913941 zero sum
0 2 306783378 zero sum
0 2 238609294 zero sum
0 2 195225786 zero sum
0 2 153391689 zero sum
0 2 119304647 zero sum
0 2 102261126 zero sum
0 2 97612893 zero sum
0 2 69273666 zero sum
0 2 65075262 zero sum
0 2 51130563 zero sum
0 2 34636833 zero sum
0 2 34087042 zero sum
0 2 32537631 zero sum
0 2 27889398 zero sum
0 2 23091222 zero sum
0 2 21691754 zero sum
0 2 17043521 zero sum
0 2 14221746 zero sum
0 2 13944699 zero sum
0 2 11545611 zero sum
0 2 10845877 zero sum
0 2 9896238 zero sum
0 2 9296466 zero sum
0 2 7697074 zero sum
0 2 7110873 zero sum
0 2 6487866 zero sum
0 2 6297606 zero sum
0 2 4948119 zero sum
0 2 4740582 zero sum
0 2 4648233 zero sum
0 2 3848537 zero sum
0 2 3298746 zero sum
0 2 3243933 zero sum
0 2 3148803 zero sum
0 2 3098822 zero sum
0 2 2370291 zero sum
0 2 2162622 zero sum
0 2 2099202 zero sum
0 2 2031678 zero sum
0 2 1649373 zero sum
0 2 1580194 zero sum
0 2 1549411 zero sum
0 2 1292886 zero sum


In conclusion, minimal data on 5 steps + 1 round is $3p^3 \approx 2^{94.6}$

# higher divisibility

In [12]:
p = 2**31-1

def model_addition_divisibility(model, u0, u1, v):
    e = model.NewBoolVar("")
    model.Add(v == u0 + u1 + e*(1-p))
    return e

def model_constant_addition_divisibility(model, u, v):
    k = model.NewIntVar(0, p-1, "")
    return model_addition_divisibility(model, u, k, v)

In [13]:
def model_round_divisibility(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    # copy left branch (branches enumerated from left to right)
    e = [model.NewIntVar(0, p-1, "") for _ in range(18)]
    cor = 0
    model_copy(model, u[0], v[2], e[0])
    model_copy(model, u[1], v[3], e[1])
    # round function
    cor += model_constant_addition_divisibility(model, e[1], e[2])
    model_copy(model, e[2], e[3], e[4]) 
    model_square(model, e[3], e[5])
    cor += model_addition_divisibility(model, e[0], e[5], e[6])
    model_copy(model, e[6], e[7], e[8]) 
    cor += model_addition_divisibility(model, e[4], e[7], e[9])
    model_copy(model, e[9], e[10], e[11]) 
    cor += model_addition_divisibility(model, e[8], e[10], e[12])
    cor += model_constant_addition_divisibility(model, e[11], e[13])
    model_copy(model, e[13], e[14], e[15]) 
    model_square(model, e[14], e[16])
    # addition to other branch
    cor += model_addition_divisibility(model, e[12], e[16], e[17])
    cor += model_addition_divisibility(model, e[17], u[3], v[1])
    cor += model_addition_divisibility(model, e[15], u[2], v[0])
    return cor

In [14]:
def model_step_divisibility(model, u, v): # we assume the variables in u and v are already restricted to 0 <= . < p
    exponents = [[model.NewIntVar(0, p-1, "") for _ in range(4)] for _ in range(4)] + [v]
    cor = 0
    # key additions
    for i in range(4):
        cor += model_constant_addition_divisibility(model, u[i], exponents[0][i])

    # rounds
    for i in range(4):
        cor += model_round_divisibility(model, exponents[i], exponents[i+1])
    return cor

In [15]:
# check positions
r = 19
nsteps = r // 4
extra_rounds = r % 4
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        cor = 0
        # input constraints
        for i in range(4):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            cor += model_step_divisibility(model, us[i], us[i+1])
        for i in range(4):
            cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        model.Minimize(cor)
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print(a, b, f"divisibility by p^{solver.ObjectiveValue()}")
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 0 divisibility by p^1.0
0 1 divisibility by p^1.0
0 2 divisibility by p^2.0
0 3 divisibility by p^1.0
1 0 divisibility by p^1.0
1 1 divisibility by p^1.0
1 2 divisibility by p^1.0
1 3 divisibility by p^1.0
2 0 divisibility by p^1.0
2 1 divisibility by p^1.0
2 2 divisibility by p^1.0
2 3 divisibility by p^1.0
3 0 divisibility by p^1.0
3 1 divisibility by p^1.0
3 2 divisibility by p^1.0
3 3 divisibility by p^1.0


In [16]:
# check positions
r = 2
nsteps = r // 4
extra_rounds = r % 4
for a in range(4):
    for b in range(4):
        model = cp_model.CpModel()
        us = [[model.NewIntVar(0, p-1, "") for _ in range(16)] for _ in range(nsteps+extra_rounds+2)]
        cor = 0
        # input constraints
        for i in range(4):
            if i == a:
                model.Add(us[0][i] == p-2)
            else:
                model.Add(us[0][i] == p-1)
            
        # model function
        for i in range(nsteps):
            cor += model_step_divisibility(model, us[i], us[i+1])
        for i in range(4):
            cor += model_constant_addition_divisibility(model, us[nsteps][i], us[nsteps+1][i])
        for i in range(extra_rounds):
            cor += model_round_divisibility(model, us[nsteps+1+i], us[nsteps+1+i+1])
        
        # output constraints
        for i in range(4):
            if i == b:
                model.Add(us[-1][i] == 1)
            else:
                model.Add(us[-1][i] == 0)
        model.Minimize(cor)
        solver = cp_model.CpSolver()
        solver.parameters.num_workers = 8
        status = solver.Solve(model)
        
        # depending on the status of the solver, print the results and a message
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            print(a, b, f"divisibility by p^{solver.ObjectiveValue()}")
        elif status == cp_model.INFEASIBLE:
            print(a, b, "zero sum")
        else:
            print(a, b, "error")

0 0 divisibility by p^3.0
0 1 divisibility by p^3.0
0 2 divisibility by p^3.0
0 3 divisibility by p^3.0
1 0 divisibility by p^3.0
1 1 divisibility by p^3.0
1 2 divisibility by p^3.0
1 3 divisibility by p^3.0
2 0 divisibility by p^3.0
2 1 divisibility by p^3.0
2 2 divisibility by p^3.0
2 3 divisibility by p^3.0
3 0 divisibility by p^3.0
3 1 divisibility by p^3.0
3 2 divisibility by p^3.0
3 3 divisibility by p^3.0
